In [ ]:
import sys
import sklearn

import tensorflow as tf
from tensorflow import keras

import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap= 'binary')
    plt.axis("off")

# PCA with a linear Autoencoder

In [ ]:
np.random.seed(42)

def generate_3d_data(m, w1=0.1, w2 =0.3, noise=0.1):
    angles= np.random.rand(m)*3*np.pi/ 2- 0.5
    data = np.empty((m,3))
    data[:,0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m)/2
    data[:,1] = np.sin(angles)* 0.7 + noise * np.random.randn(m)/2
    data[:,2] = data[:,0] * w1 + data[:,1] * w2 + noise * np.random.randn(m)
    return data

X_train = generate_3d_data(60)
X_train = X_train - X_train.mean(axis=0, keepdims=0)
    

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

encoder = keras.model.Sequential([keras.layers.Dense(2, input_shape=[3])])
decoder = keras.model.Sequential([keras.layers.Dense(3, input_shape=[2])])
autoencoder = keras.model.Sequential([encoder, decoder])

autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1.5))

In [ ]:
history=autoencoder.fit(X_train, X_train, epochs=20)

In [ ]:
codings=encoder.predict(X_train)

In [ ]:
fig=plt.figure(figsize=(4,3))
plt.plot(codings[:,0], codings[:,1], "b.")
plt.xlabel("$z_1$", fontsize=18)
plt.ylabel("$z_2$", fontsize=18, rotation=0)
plt.grid(True)

# Stacked Autoencoder

In [ ]:
(X_train_full, y_train_full), (X_test, y_test)=keras.datasets.fashion_mnist.load_data()
X_train_full=X_train_full.astype(np.float32)/255
X_test =X_test.astype(np.float32)/255
X_train, X_valid=X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid=y_train_full[:-5000], y_train_full[-5000:]

## Train all layers at once

In [ ]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape[28,28]),
    keras.layers.Dense(100, activation= 'selu'),
    keras.layers.Dense(30, activation= 'selu')])

stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation= 'selu', input_shape=[30]),
    keras.layers.Dense(28*28, activation= 'sigmoid'),
    keras.layers.Reshape([28,28])])

stacked_ae=keras.models.Sequential([stacked_encoder, stacked_decoder])
stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=1.5), metrics=[rounded_accuracy])
history=stacked_ae.fit(X_train, X_train, epochs=20, validation_data=(X_valid, X_valid))

In [ ]:
def show_reconstructions(model, images=X_valid, n_images=5):
    reconstructions=model.predict(images[:n_images])
    fig=plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1+image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1+n_images+image_index)
        plot_image(reconstructions[image_index])

In [ ]:
show_reconstructions(stacked_ae)

# Visualizing Fashion MNIST

In [ ]:
np.random.seed(42)

from sklearn.manifold import TSNE

X_valid_compressed=stacked_encoder.predict(X_valid)
tsne=TSNE()
X_valid_2D=tsne.fit_transform(X_valid_compressed)
X_valid_2D=(X_valid_2D - X_valid_2D.min())/(X_valid_2D.max() - X_valid_2D.min())

In [ ]:
plt.scatter(X_valid_2D[:,0], X_valid_2D[:,1], c=y_valid, s=10, cmap="tab10")
plt.axis('off')

In [ ]:
plt.figure(figsize=(10,8))
cmap=plt.com.tab10
plt.scatter(X_valid_2D[:,0], X_valid_2D[:,1], c= y_valid, s=10, cmap=cmap)
image_positions=np.array([[1.,1.]])
for index, position in enumerate(X_valid_2D):
    dist=np.sum((position - image_positions)** 2, axis=1)
    if np.min(dist) > 0.02:
        image_positions=np.r_[image_positions, [position]]
        imagebox=mpl.offsetbox.AnnotationBbox(
            mpl.offsetbox.OffsetImage(X_valid[index], cmap="binary"),
            position, bboxprops={'edgecolor':cmap(y_valid[index]), "lw":2})
        plt.gca().add_artist(imagebox)
plt.axis('off')


## Tying weights